In [1]:
import datasets
import torch
import transformers

In [2]:
dataset = datasets.load_dataset("gsm8k", "main", split="test")

Found cached dataset gsm8k (/home/mila/g/gagnonju/.cache/huggingface/datasets/gsm8k/main/1.1.0/37bfb08b1d4fcbb01f06b03d9e1ef5f1fcbd4d3af3d08842c50d7305091285ba)


In [3]:
tokenizer = transformers.AutoTokenizer.from_pretrained("google/flan-t5-xxl")

In [4]:
lengths_questions = [len(tokenizer(x["question"])["input_ids"]) for x in dataset]

In [5]:
print(len(lengths_questions))
print(len([x for x in lengths_questions if x < 100]))
print(f"{len([x for x in lengths_questions if x < 100]) / len(lengths_questions):0.1%}")

1319
1233
93.5%


In [6]:
lengths_answers = sorted([len(tokenizer(x["answer"])["input_ids"]) for x in dataset])
print(len(lengths_answers))
limit = 192

print(len([x for x in lengths_answers if x < limit]))
print(f"{len([x for x in lengths_answers if x < limit]) / len(lengths_answers):0.1%}")


1319
1264
95.8%


In [7]:
questions = [len(tokenizer(x["question"])["input_ids"]) for x in dataset]
answers   = [len(tokenizer(x["answer"  ])["input_ids"]) for x in dataset]

In [8]:
max_len_questions = 83
max_len_answers   = 192
print(f"{len([None for q, a in zip(questions, answers) if q < max_len_questions and a < max_len_answers]) / len(dataset):0.1%}")


82.6%


In [9]:
# upper = pow(0.9223, 1 / 3)
upper = .97
assert len(questions) == len(answers)
total_l = len(questions)

limit_pos = int(upper * total_l)

max_encoder_l = sorted(questions)[limit_pos]
max_decoder_l = sorted(answers)[limit_pos]
max_squared_sum_l = sorted([(i ** 2 + j ** 2, i, j) for i, j in zip(questions, answers)], key=lambda x: x[0])[limit_pos][0]
print(
    f"upper: {upper:0.2%},\n"
    f"max_encoder_l: {max_encoder_l},\n"
    f"max_decoder_l: {max_decoder_l},\n"
    f"max_squared_sum_l: {max_squared_sum_l}"
)
filtered = [(i, j) for i, j in zip(questions, answers) if i < max_encoder_l and j < max_decoder_l and i ** 2 + j ** 2 < max_squared_sum_l]
print(f"{len(filtered) / len(dataset):0.1%}")

upper: 96.00%,
max_encoder_l: 112,
max_decoder_l: 192,
max_squared_sum_l: 45889
92.9%


In [10]:
filtered_ds = dataset.filter(lambda x: len(tokenizer(x["question"])["input_ids"]) ** 2 + len(tokenizer(x["answer"])["input_ids"]) ** 2 < 43205)

Loading cached processed dataset at /home/mila/g/gagnonju/.cache/huggingface/datasets/gsm8k/main/1.1.0/37bfb08b1d4fcbb01f06b03d9e1ef5f1fcbd4d3af3d08842c50d7305091285ba/cache-2e7168e5481a8abc.arrow


In [11]:
f"{len(filtered_ds) / len(dataset):0.1%}"

'95.4%'